# Assignment 3 Question 3
### Lokesh Chandra Das

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense,Flatten,Dropout, BatchNormalization,Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import initializers
import os
import cv2




from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import Counter
DATASET_PATH = '/content/drive/MyDrive/Dataset/15-Scene/'
directories = os.listdir(DATASET_PATH)
ONE_HOT_ENCODED = np.eye(len(directories))
FEATURES = []
LABELS = []
for category in sorted(directories):
    path = DATASET_PATH+category
    files = os.listdir(path)
    for fileName in files:
        imgPath = path+"/"+fileName
        img = cv2.imread(imgPath)
        img = cv2.resize(img, (32,32))
        #img = np.reshape(img,[32,32,3])
        FEATURES.append(img)
        LABELS.append((np.reshape(ONE_HOT_ENCODED[int(category)], [15])))

In [ ]:
# Randomly shuffle the datasets
shuffle_indices = np.random.permutation(len(FEATURES))
FEATURES = np.array(FEATURES)
LABELS = np.array(LABELS)
FEATURES = FEATURES[shuffle_indices]
LABELS = LABELS[shuffle_indices]

In [ ]:
TEST_PERCENTAGE = 0.3 # 30% for testing and 70% for training
train_X = FEATURES[:int(len(FEATURES)*(1-TEST_PERCENTAGE))]
train_y = LABELS[:int(len(LABELS)*(1-TEST_PERCENTAGE))]
print(train_X.shape)

(3139, 32, 32, 3)


In [ ]:
test_X = FEATURES[int(len(FEATURES)*(1-TEST_PERCENTAGE)):]
test_y = LABELS[int(len(LABELS)*(1-TEST_PERCENTAGE)):]
print(test_X.shape)

(1346, 32, 32, 3)


In [ ]:
isAugmentation = True
activation='elu'
batch_size = 64
epochs = 150
num_classes = 15
INITIALIZER = initializers.GlorotNormal() # xavier initializer
SAVE_DIR = os.path.join(os.getcwd(),'models')
MODEL_NAME='ASSIGNMENT_2_PART_2.h5'

In [ ]:
train_X = train_X.astype('float32')
train_y = train_y.astype('float32')
test_X = test_X.astype('float32')
test_y = test_y.astype('float32')
train_X /= 255.0
test_X /= 255.0

In [ ]:
def dataAugmentation(isAugmentation=False):
    if not isAugmentation:
        return None
    else:
        dataGenerator =  ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.1)
        return dataGenerator
        

In [ ]:
dataGenerator = dataAugmentation(True)

# Optimizer RMSProp


In [ ]:
def createModel():

  weight_decay = 1e-6
  model = Sequential()
  model.add(Conv2D(32, (5,5), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=train_X.shape[1:]))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, (5,5), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=INITIALIZER, kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Activation('elu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.45))
  
  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))
  optRMSprop = opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=optRMSprop, metrics=['accuracy'])
  return model


In [ ]:
indModel = createModel()
if dataGenerator is not None:
  dataGenerator.fit(train_X)
  ind_history = indModel.fit(dataGenerator.flow(train_X, train_y,
                                     batch_size=batch_size,subset='training'), 
                        validation_data=dataGenerator.flow(train_X, train_y,
                                     batch_size=batch_size,subset='validation'),verbose=2,epochs=epochs)
else:
 ind_history = indModel.fit(train_X,train_y, batch_size=batch_size,epochs=epochs,validation_data=(test_X,test_y),verbose=2)

Epoch 1/150
45/45 - 19s - loss: 3.7039 - accuracy: 0.1783 - val_loss: 4.4154 - val_accuracy: 0.0990
Epoch 2/150
45/45 - 1s - loss: 2.8473 - accuracy: 0.2590 - val_loss: 5.3169 - val_accuracy: 0.1693
Epoch 3/150
45/45 - 1s - loss: 2.3992 - accuracy: 0.3524 - val_loss: 7.5949 - val_accuracy: 0.0863
Epoch 4/150
45/45 - 1s - loss: 2.1677 - accuracy: 0.3903 - val_loss: 7.0604 - val_accuracy: 0.1534
Epoch 5/150
45/45 - 1s - loss: 1.9505 - accuracy: 0.4490 - val_loss: 9.0020 - val_accuracy: 0.1278
Epoch 6/150
45/45 - 1s - loss: 1.8549 - accuracy: 0.4710 - val_loss: 5.3661 - val_accuracy: 0.2268
Epoch 7/150
45/45 - 1s - loss: 1.7175 - accuracy: 0.5032 - val_loss: 3.4875 - val_accuracy: 0.3578
Epoch 8/150
45/45 - 1s - loss: 1.6332 - accuracy: 0.5156 - val_loss: 3.2525 - val_accuracy: 0.3323
Epoch 9/150
45/45 - 1s - loss: 1.6100 - accuracy: 0.5180 - val_loss: 2.1850 - val_accuracy: 0.4409
Epoch 10/150
45/45 - 1s - loss: 1.4909 - accuracy: 0.5333 - val_loss: 2.4329 - val_accuracy: 0.4537
Epoch 11

In [ ]:
evaluations = indModel.evaluate(test_X,test_y)
print(evaluations)

43/43 [==============================] - 0s 4ms/step - loss: 1.3057 - accuracy: 0.7333
[1.305678129196167, 0.7332838177680969]


In [ ]:
print('Single Model Accuracy: ', np.around((evaluations[1]*100),decimals=2)," %")

Single Model Accuracy:  73.33  %


In [ ]:
model = createModel()
num_of_model_to_ensemble = 3

In [ ]:
def getModel():
  allTrainedModels = []
  for _ in range(num_of_model_to_ensemble):
    model = createModel()
    if dataGenerator is not None:
      dataGenerator.fit(train_X)
      model.fit(dataGenerator.flow(train_X, train_y,
                                     batch_size=batch_size,subset='training'), 
                        validation_data=dataGenerator.flow(train_X, train_y,
                                     batch_size=batch_size,subset='validation'),verbose=2,epochs=epochs)
    else:
      model.fit(train_X,train_y, batch_size=batch_size,epochs=epochs,validation_data=(test_X,test_y),verbose=2)
    allTrainedModels.append(model)
  return allTrainedModels

In [ ]:
ensembleModels = getModel()

Epoch 1/150
45/45 - 3s - loss: 3.7718 - accuracy: 0.1681 - val_loss: 3.9194 - val_accuracy: 0.1278
Epoch 2/150
45/45 - 1s - loss: 2.9265 - accuracy: 0.2608 - val_loss: 6.7025 - val_accuracy: 0.1054
Epoch 3/150
45/45 - 1s - loss: 2.5341 - accuracy: 0.3089 - val_loss: 9.2348 - val_accuracy: 0.0895
Epoch 4/150
45/45 - 1s - loss: 2.2191 - accuracy: 0.3740 - val_loss: 7.9683 - val_accuracy: 0.1022
Epoch 5/150
45/45 - 1s - loss: 2.1204 - accuracy: 0.3999 - val_loss: 7.0990 - val_accuracy: 0.1118
Epoch 6/150
45/45 - 1s - loss: 1.9077 - accuracy: 0.4420 - val_loss: 9.2660 - val_accuracy: 0.1310
Epoch 7/150
45/45 - 1s - loss: 1.7983 - accuracy: 0.4636 - val_loss: 3.6065 - val_accuracy: 0.2716
Epoch 8/150
45/45 - 1s - loss: 1.6776 - accuracy: 0.5096 - val_loss: 3.0626 - val_accuracy: 0.3482
Epoch 9/150
45/45 - 1s - loss: 1.5803 - accuracy: 0.5234 - val_loss: 2.6011 - val_accuracy: 0.3994
Epoch 10/150
45/45 - 1s - loss: 1.4987 - accuracy: 0.5333 - val_loss: 1.9541 - val_accuracy: 0.4792
Epoch 11/

In [ ]:
def ensemblePrediction(models):
  predictions = [model.predict(test_X) for model in models ]
  predictions = np.array(predictions)
  summed = np.sum(predictions,axis=0)
  result = np.argmax(summed,axis=1)
  return result

In [ ]:
prediction = ensemblePrediction(ensembleModels)
test_y_arg=np.argmax(test_y,axis=1)
print('Ensemble accuracy:', np.around(accuracy_score(prediction,test_y_arg),decimals=2)*100,"%")

Ensemble accuracy: 77.0 %
